# Residualer

Vi fortsætter med at betragte datasættet

$$

    (x_1,y_1),(x_2,y_2),\dots,(x_n,y_n),

$$

sammen med vores simple lineære model

$$

    f(x) = \beta_0 + \beta_1 x,

$$

*fitted* på vores data. I forrige afsnit kiggede vi på, hvordan vi kunne bruge de lodrette afstande fra vores datapunkter til linjen til at finde den rette linje, der bedst gik gennem vores datapunkter. Disse lodrette afstande kaldes for *residualer*. Vi vil benævne dem med $z_i$ for den $i$'te observation og de er givet som

$$

    z_i = y_i - f(x_i).

$$ (eq:residualer)

In [4]:
from manim import *
import numpy as np
from scipy.stats import norm, uniform

In [5]:
%%manim -v WARNING -qm --format=mp4 SpecificPoint

# Config manim
config.media_embed = True
config.media_width = "100%"

rng = np.random.default_rng(seed=5)
x_data = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=float)
y_data = 2 * x_data + 2 + 1.5 * norm.rvs(size=len(x_data), random_state=rng)

animat_blue   = "#6a90cc"
animat_red    = "#cc5241"
animat_green  = "#aecc55"
animat_yellow = "#d9c750"

class SpecificPoint(MovingCameraScene):
    def construct(self):
        self.camera.background_color = WHITE

        # 1) Create Axes
        ax = Axes(
            x_range=[0, np.max(x_data) + 2, 2],
            y_range=[0, np.max(y_data) + 2, 2],
            axis_config={"include_numbers": True, "color": BLACK},
        )
        labels = ax.get_axis_labels(
            x_label=MathTex("x", color=BLACK),
            y_label=MathTex("y", color=BLACK)
        )
        self.add(ax, labels)

        points = VGroup(*[
            Dot(ax.c2p(xv, yv), color=animat_blue)
            for xv, yv in zip(x_data, y_data)
        ])
        points.set_z_index(3)
        self.add(points)

        # 2) Compute a simple line (like a rough fit)
        min_x, max_x = np.min(x_data), np.max(x_data)
        y_minx = y_data[np.argmin(x_data)]
        y_maxx = y_data[np.argmax(x_data)]
        slope = (y_maxx - y_minx) / (max_x - min_x)
        intercept = y_maxx - slope * max_x
        
        # 3) Find the point with largest vertical distance to that line
        line_vals = slope * x_data + intercept
        diffs = np.abs(y_data - line_vals)
        i_max = np.argmax(diffs)
        x0, y0 = x_data[i_max], y_data[i_max]
        y_on_line = line_vals[i_max]

        # 4) Force the camera to start zoomed in on (x0,y0)
        self.camera.frame.scale(0.3)
        self.camera.frame.move_to(ax.c2p(x0, y0))

        # 5) Now draw everything in the zoomed view:
        fit_line = ax.plot(lambda xx: slope*xx + intercept, color=animat_green)
        self.play(Create(fit_line))

        f_label = MathTex(r"f", color=animat_green, font_size=14)
        f_label.next_to(fit_line.get_end(), RIGHT)
        self.play(Write(f_label))

        point_of_interest = Dot(ax.c2p(x0, y0), color=animat_blue)
        point_of_interest.set_z_index(3)
        #self.play(Indicate(point_of_interest))

        y_label = MathTex(r"y_i", color=BLACK, font_size=14)
        y_label.next_to(ax.c2p(x0, y0), DOWN, buff=0.1)
        self.play(Write(y_label))

        line_point = Dot(ax.c2p(x0, y_on_line), color=BLACK)
        line_point.set_z_index(3)
        self.play(FadeIn(line_point))
        
        fx_label = MathTex(r"f(x_i)", color=BLACK, font_size=14)
        fx_label.next_to(ax.c2p(x0, y_on_line), UP, buff=0.1)
        self.play(Write(fx_label))

        vertical_line = DashedLine(
            start=ax.c2p(x0, y0),
            end=ax.c2p(x0, y_on_line),
            color=animat_red
        )
        vertical_line.set_z_index(1)

        eq = MathTex(
            r"z_i", "=", r"y_i", "-", r"f(x_i)",
            color=BLACK, font_size=14
        )
        eq[0].set_color(animat_red)
        eq.next_to(vertical_line, RIGHT, buff=0.1)

        self.play(
            Create(vertical_line),
            Write(eq[0])
        )

        self.play(Write(eq[1]))             # =
        self.play(Indicate(point_of_interest))  # highlight y_i
        self.play(Write(eq[2]))             # y_i
        self.play(Write(eq[3]))             # minus
        self.play(Indicate(line_point))     # highlight f(x_i)
        self.play(Write(eq[4]))             # f(x_i)

        self.wait(4)


Omformer vi {eq}`eq:residualer`, så har vi at

$$

    y_i = f(x_i) + z_i.

$$ (eq:model-med-residualer)

Ligning {eq}`eq:model-med-residualer` fortæller os, at vi kan opdele vores observationer i to komponenter:

1. $f(x_i)$: Dette er vores prædikterede værdi fra vores model $f(x) = \beta_0 + \beta_1 x$. Dette er punktet på linjen, som bedst beskriver sammenhængen mellem $x_i$ og $y_i$ ifølge vores model. 
2. $z_i$: Dette er vores residual (eller fejlen). Det er forskellen mellem den faktisk observerede værdi $y_i$ og den prædikterede værdi $f(x_i)$. Vi kalder det også fejlen, da den angiver hvor meget vores model afviger fra den faktiske observation.

Denne opdeling er vigtig, da den viser, at hver observation $y_i$ består af den del, der kan forklares af den lineære model $f(x)$, og den del, der er uforklaret variation (residualerne $z_i$). 

For at forstå dette bedre kan du tænke på følgende: Vores datasæt kan være baseret på en lineær sammenhæng, der beskrives af forskriften $f(x)=\beta_0 + \beta_1 x$. Men i praksis vil der være *støj* eller tilfældige afvigelser i dataene, hvilket medfører, at de faktiske observationer $y_i$ ikke ligger præcist på den forudsagte linje. Denne støj er det, som residualerne fanger. Residualerne afspejler altså den uforklarede variation eller de små afvigelser fra den ideelle lineære sammenhæng, som skyldes støj, målefejl eller andre faktorer, der ikke er fanget af modellen. 

## Residualernes vigtige rolle

Residualerne er helt centrale, når det kommer til at skulle evaluere, hvor god vores model er på vores data. Vi kræver en bestemt opførsel af residualerne for at konkludere, at en model er god til at beskrive sammenhængen i vores data.

For at vurdere residualernes opførsel kigger vi ofte i et residualplot. Et residualplot er blot et scatterplot, hvor vi plotter residualerne $z_i$ mod $x_i$ for alle vores datapunkter $i=1,2,\dots,n$.


### Residualerne skal være så små som muligt

Dette er nærmest en selvfølge. Husk, residualerne kaldes også fejlen. Vi ønsker, at forskellen mellem de faktisk observerede værdier $y_i$ og de prædikterede værdier $f(x_i)$ er minimal. Desto mindre fejlen er, desto tættere på modellen befinder vores faktiske værdier sig. Det var præcis det idéen var bag mindste kvadraters metode. Her så vi, hvordan vi ved at minimere summen af kvadraterne på de lodrette afstande mellem datapunkterne og linjen, kunne bestemme den linje, der bedst gik gennem vores data. Disse lodrette afstande er jo netop residualerne. Med andre ord, så går mindste kvadraters metode ud på at minimere summen af kvadraterne på residualerne. Det kan vi også se, hvis vi indsætter {eq}`eq:residualer` i {eq}`eq:mindste-kvadraters-metode`

$$

    \text{RSS} = \sum_{i=1}^n (y_i - f(x_i))^2 = \sum_{i=1}^n z_i^2.

$$

I simpel lineær regression er det denne metode, som vi bruger.

In [2]:
from manim import *

Manim Community v0.18.0.post0

In [3]:
%%manim -v WARNING -qm --format=mp4 S1_MinimizeResiduals

# Config manim
config.media_embed = True
config.media_width = "100%"

animat_green = "#aecc55"
animat_red = "#cc5241"
animat_yellow = "#d9c750"
animat_blue = "#6a90cc"

class S1_MinimizeResiduals(Scene):
    def construct(self):
        self.camera.background_color = WHITE  # Hvid baggrund

        # --------------------------------------------------------
        # 1) Definér data
        # --------------------------------------------------------
        x_data = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=float)
        y_data = np.array([1.2, 2.0, 2.3, 3.8, 5.1, 5.0, 6.2, 7.1, 7.5, 8.9], dtype=float)

        # "Rigtige" (optimale) slope/intercept via polyfit
        slope_true, intercept_true = np.polyfit(x_data, y_data, 1)

        # "Dårlig" start
        slope_bad = 3.0
        intercept_bad = 0.0

        slope_tracker   = ValueTracker(slope_bad)
        intercept_tracker = ValueTracker(intercept_bad)

        # --------------------------------------------------------
        # 2) Opret akser (top + bund), små ticks, tips, "z"-label
        # --------------------------------------------------------
        x_min, x_max = min(x_data) - 0.5, max(x_data) + 0.5
        x_range = [x_min, x_max+0.5, 1]
        y_range_top = [0, 10, 1]
        y_range_bot = [-3, 3, 1]

        axis_config = {
            "color": BLACK,
            "include_numbers": True,
            "include_ticks": True,
            "tick_size": 0.05,  # små ticks
            "stroke_width": 1,  # tynde akser
        }

        top_axes = Axes(
            x_range=x_range,
            y_range=y_range_top,
            x_length=6,
            y_length=3,
            tips=True,  # Pile-ender
            axis_config=axis_config
        ).to_edge(UP)

        bot_axes = Axes(
            x_range=x_range,
            y_range=y_range_bot,
            x_length=6,
            y_length=2,
            tips=True,  # Pile-ender
            axis_config=axis_config
        ).next_to(top_axes, DOWN, buff=1.0)

        top_axes_labels = top_axes.get_axis_labels(
            x_label=MathTex("x", color=BLACK).scale(0.7),
            y_label=MathTex("y", color=BLACK).scale(0.7)
        )
        bot_axes_labels = bot_axes.get_axis_labels(
            x_label=MathTex("x", color=BLACK).scale(0.7),
            y_label=MathTex("z", color=BLACK).scale(0.7)  # Residual = z
        )

        self.add(top_axes, bot_axes, top_axes_labels, bot_axes_labels)

        # --------------------------------------------------------
        # 3) Definér en offset_tracker til *senere* brug (punkter løftes)
        # --------------------------------------------------------
        # Vi venter med at animere den til sidst.
        offset_tracker = ValueTracker(0.0)

        # Lav et array af random offsets, så nogle punkter går op, andre ned
        # For en pæn demonstration, lad os tage typisk +/-1.5
        rng = np.random.default_rng(2023)  # fast seed
        random_offsets = rng.uniform(-1.5, 1.5, size=len(x_data))

        def get_adjusted_y(i):
            """
            Returnerer den aktuelle y-værdi for punkt i,
            efter at offset_tracker er taget i betragtning.
            """
            base_y = y_data[i]
            alpha  = offset_tracker.get_value()
            # tilføj alpha*gange den random offset:
            return base_y + alpha * random_offsets[i]


        # --------------------------------------------------------
        # 4) Plot datapunkter i top (nu med always_redraw)
        # --------------------------------------------------------
        # I stedet for at "bare" lægge dem i scenen, laver vi en
        # always_redraw-funktion for hver dot, så de kan bevæge sig.
        scatter_dots = VGroup()
        for i, (x, y) in enumerate(zip(x_data, y_data)):
            def make_top_dot(i=i):
                x_val = x_data[i]
                y_val = get_adjusted_y(i)  # <-- justeret y
                return Dot(
                    point=top_axes.coords_to_point(x_val, y_val),
                    color=animat_blue
                )
            dot = always_redraw(make_top_dot)
            scatter_dots.add(dot)

        self.add(scatter_dots)  # ingen transition her, men vi kunne bruge Create()

        # --------------------------------------------------------
        # 5) Dynamisk linje (afhænger af slope_tracker og intercept_tracker)
        # --------------------------------------------------------
        def line_mob():
            return top_axes.plot(
                lambda xx: slope_tracker.get_value()*xx + intercept_tracker.get_value(),
                x_range=[x_data[0], x_data[-1]],
                color=animat_green
            )
        line = always_redraw(line_mob)
        self.play(Create(line))

        # --------------------------------------------------------
        # 6) Lodrette residual-linjer (top-plot) always_redraw
        # --------------------------------------------------------
        residual_lines = VGroup()
        for i, (x, y) in enumerate(zip(x_data, y_data)):
            def make_line(i=i):
                m = slope_tracker.get_value()
                b = intercept_tracker.get_value()
                x_val = x_data[i]
                # y justeret
                y_val = get_adjusted_y(i)
                yhat = m*x_val + b
                return Line(
                    start=top_axes.coords_to_point(x_val, y_val),
                    end=top_axes.coords_to_point(x_val, yhat),
                    color=animat_red
                )
            line_obj = always_redraw(make_line)
            residual_lines.add(line_obj)

        # --------------------------------------------------------
        # 7) Residual-dots i bunden (always_redraw)
        # --------------------------------------------------------
        residual_dots = VGroup()
        for i, (x, y) in enumerate(zip(x_data, y_data)):
            def make_bot_dot(i=i):
                m = slope_tracker.get_value()
                b = intercept_tracker.get_value()
                x_val = x_data[i]
                y_val = get_adjusted_y(i)  # justeret
                z = y_val - (m*x_val + b)
                return Dot(bot_axes.coords_to_point(x_val, z), color=animat_red)
            dot_obj = always_redraw(make_bot_dot)
            residual_dots.add(dot_obj)

        zero_line = bot_axes.plot(lambda xx: 0, x_range=[x_data[0], x_data[-1]], color=GRAY)

        self.play(
            LaggedStartMap(Create, residual_lines, lag_ratio=0.1),
            LaggedStartMap(Create, residual_dots, lag_ratio=0.1),
            #Create(zero_line)
        )

        # --------------------------------------------------------
        # 8) Dynamisk RSS-tekst (always_redraw)
        # --------------------------------------------------------
        def rss_text_mob():
            m = slope_tracker.get_value()
            b = intercept_tracker.get_value()
            # Nu skal vi bruge de justerede y-værdier i RSS:
            rss_val = 0
            for i in range(len(x_data)):
                x_i = x_data[i]
                y_i = get_adjusted_y(i)
                rss_val += (y_i - (m*x_i + b))**2
            txt = MathTex(
                r"\text{RSS} = ",
                f"{rss_val:.2f}",  # 2 decimaler
                color=BLACK
            )
            txt.scale(0.7)
            txt.to_corner(UR)
            return txt

        rss_dynamic_text = always_redraw(rss_text_mob)
        self.play(FadeIn(rss_dynamic_text))

        # self.play(Circumscribe(rss_dynamic_text, fade_out=True, color=animat_red))
        # Lav en highlight-boks om RSS
        highlight_box = SurroundingRectangle(rss_dynamic_text, color=animat_red, buff=0.2)
        self.play(Create(highlight_box))

        minimize_this_text = Tex(r"Find den linje, \\ der minimerer RSS", font_size=24, color=BLACK).move_to(rss_dynamic_text.get_center()+1*DOWN)
        self.play(Write(minimize_this_text))
        self.wait(1)

        # --------------------------------------------------------
        # 9) Animer slope/intercept -> "bedste"
        # --------------------------------------------------------
        self.play(
            slope_tracker.animate.set_value(slope_true),
            intercept_tracker.animate.set_value(intercept_true),
            FadeOut(highlight_box, lag_ratio=2),
            run_time=4
        )

        self.play(Circumscribe(rss_dynamic_text, fade_out=True, color=animat_red), Indicate(line, color=animat_red), run_time=2)
        self.wait(1)

        self.play(FadeOut(minimize_this_text))
        self.wait(1)

        describing_text = Tex(r"Når punkterne er længere væk \\ fra regressionslinjen, stiger RSS", font_size=24, color=BLACK).move_to(scatter_dots[9].get_center()+1.4*DOWN)
        arrow_to_rss = Arrow(start=describing_text.get_top(), end=rss_dynamic_text[1].get_bottom(), buff=0.2, color=BLACK, stroke_width=1.1, max_tip_length_to_length_ratio=0.04)
        
        self.play(Write(describing_text))
        self.play(Create(arrow_to_rss))
        self.wait(1)

        # --------------------------------------------------------
        # 10) NY DEL: Ryk punkterne lodret væk (offset_tracker: 0->1), 
        #     se RSS stige, og så tilbage (offset_tracker: 1->0)
        # --------------------------------------------------------
        # Bemærk: Den fittede linje er *uændret* (slope_true, intercept_true).

        # 10a) Ryk væk (0 -> 1)
        self.play(offset_tracker.animate.set_value(1), run_time=3)
        self.wait(2)

        # 10b) Ryk tilbage (1 -> 0)
        self.play(offset_tracker.animate.set_value(0), run_time=3)
        self.play(FadeOut(describing_text),FadeOut(arrow_to_rss))
        self.wait(4)

        

### Residualerne skal være tilfældige

Vi ønsker ikke, at der er et systematisk mønster i residualerne. Dette undersøger vi ved en visuel inspektion af residualerne i et residualplot. Hvis der er et mønster i residualplottet, betyder det, at vores model ikke fanger alle strukturerne i dataene. Det vil oftest betyde, at vi har brug for en mere kompleks model.

In [8]:
from manim import *

In [9]:
%%manim -v WARNING -qm --format=mp4 S2_RandomResiduals

# Config manim
config.media_embed = True
config.media_width = "100%"

class S2_RandomResiduals(Scene):
    def construct(self):
        self.camera.background_color = WHITE

        # --------------------------------------------------------
        # 1) Data & Models: M0, M1, M2
        # --------------------------------------------------------
        np.random.seed(42)
        x_data = np.linspace(0, 10, 20)
        noise = np.random.normal(0, 1.0, len(x_data))
        # We'll use a parabolic "true" data for top
        y_data_poly = 0.25 * x_data**2 + 2.0 + noise

        # Three model stages: each is (a,b,c) for a*x^2 + b*x + c
        # We'll transition alpha: 0→1→2→3 piecewise.
        best_slope, best_intercept = np.polyfit(x_data, y_data_poly, 1)

        M0 = (0.0, best_slope, best_intercept)  # near "best" line
        M1 = (0.0, 1.5,        3.0)            # a simple line
        M2 = (0.25, 0.0,       2.0)            # a parabolic

        # --------------------------------------------------------
        # 2) Axes
        # --------------------------------------------------------
        x_min, x_max = 0, 10
        x_range = [x_min - 0.5, x_max + 0.5, 2]
        y_range_top = [0, 25, 2]
        y_range_bot = [-5, 5, 1]

        axis_config = dict(
            color=BLACK,
            include_numbers=True,
            include_ticks=True,
            tick_size=0.05,
            stroke_width=1,
        )

        top_axes = Axes(
            x_range=x_range, y_range=y_range_top,
            x_length=6,      y_length=3,
            tips=True,
            axis_config=axis_config
        ).to_edge(UP)

        bot_axes = Axes(
            x_range=x_range, y_range=y_range_bot,
            x_length=6,      y_length=2,
            tips=True,
            axis_config=axis_config
        ).next_to(top_axes, DOWN, buff=1.0)

        top_labels = top_axes.get_axis_labels(
            MathTex("x", color=BLACK).scale(0.7),
            MathTex("y", color=BLACK).scale(0.7)
        )
        bot_labels = bot_axes.get_axis_labels(
            MathTex("x", color=BLACK).scale(0.7),
            MathTex("z", color=BLACK).scale(0.7)  # residual
        )

        self.add(top_axes, bot_axes, top_labels, bot_labels)

        # --------------------------------------------------------
        # 3) Plot data points (top)
        # --------------------------------------------------------
        scatter_dots = VGroup(*[
            Dot(top_axes.coords_to_point(x, y), color=animat_blue)
            for x, y in zip(x_data, y_data_poly)
        ])
        self.add(scatter_dots)

        # --------------------------------------------------------
        # 4) Top model: M0->M1->M2->M0 with alpha
        # --------------------------------------------------------
        alpha_tracker = ValueTracker(0.0)

        def paramSet(alpha):
            """
            Piecewise interpolation in [0..1..2..3]
            for (a,b,c).
            """
            (a0,b0,c0) = M0
            (a1,b1,c1) = M1
            (a2,b2,c2) = M2
            if 0 <= alpha <= 1:
                t = alpha
                return (a0*(1 - t) + a1*t,
                        b0*(1 - t) + b1*t,
                        c0*(1 - t) + c1*t)
            elif 1 < alpha <= 2:
                t = alpha - 1
                return (a1*(1 - t) + a2*t,
                        b1*(1 - t) + b2*t,
                        c1*(1 - t) + c2*t)
            else:  # 2 < alpha <= 3
                t = alpha - 2
                return (a2*(1 - t) + a0*t,
                        b2*(1 - t) + b0*t,
                        c2*(1 - t) + c0*t)

        def top_model_func(x):
            a,b,c = paramSet(alpha_tracker.get_value())
            return a*x**2 + b*x + c

        # We'll draw the top model so we can see how M0->M1->M2->M0 changes
        model_curve = always_redraw(
            lambda: top_axes.plot(
                lambda xx: top_model_func(xx),
                x_range=[x_min, x_max],
                color=animat_green
            )
        )
        self.play(Create(model_curve))

        # --------------------------------------------------------
        # 5) Residual lines + dots in bottom
        # --------------------------------------------------------
        residual_lines = VGroup()
        residual_dots  = VGroup()

        for x, y in zip(x_data, y_data_poly):
            def make_line(x=x, y=y):
                a,b,c = paramSet(alpha_tracker.get_value())
                yhat  = a*x**2 + b*x + c
                return Line(
                    start=top_axes.coords_to_point(x,y),
                    end=top_axes.coords_to_point(x,yhat),
                    color=animat_red
                )
            residual_lines.add(always_redraw(make_line))

            def make_dot(x=x, y=y):
                a,b,c = paramSet(alpha_tracker.get_value())
                yhat  = a*x**2 + b*x + c
                z     = y - yhat
                return Dot(bot_axes.coords_to_point(x, z), color=animat_red)
            residual_dots.add(always_redraw(make_dot))

        self.play(
            LaggedStartMap(Create, residual_lines, lag_ratio=0.1),
            LaggedStartMap(Create, residual_dots,  lag_ratio=0.1)
        )

        # --------------------------------------------------------
        # 6) 2nd-order polynomial fit of the residuals at each alpha
        #    We'll only "flash" it. No permanent curve.
        # --------------------------------------------------------
        # Compute polynomial of degree=2 for the residuals at a certain alpha
        def residual_poly_deg2(xdata, ydata, a, b, c):
            # z_i = y_i - model(x_i)
            zvals = [ (yi - (a*xi**2 + b*xi + c)) for (xi, yi) in zip(xdata, ydata) ]
            return np.polyfit(xdata, zvals, 2)  # returns [p2, p1, p0]

        # This function returns [p2, p1, p0] for the "residual fit" at alpha
        def get_residual_poly(alpha):
            (a,b,c) = paramSet(alpha)
            return residual_poly_deg2(x_data, y_data_poly, a, b, c)

        # A function to evaluate the residual curve at alpha
        def residual_curve(alpha, x):
            p2, p1, p0 = get_residual_poly(alpha)
            return p2*x**2 + p1*x + p0

        # --------------------------------------------------------
        # 8) Helper to flash the 2nd-order residual polynomial
        #    *without* leaving a visible curve
        # --------------------------------------------------------
        def flash_residual_poly(at_alpha, flash_color=animat_green, width=4):
            """
            1) Create a static 2nd-order polynomial for the residuals at 'at_alpha'.
            2) ShowPassingFlash with big stroke.
            3) Remove the static curve so it doesn't remain.
            """
            # Build that polynomial as a VMobject:
            static_curve = bot_axes.plot(
                lambda x: residual_curve(at_alpha, x),
                x_range=[x_min, x_max],
            )
            # Animate a flash
            self.play(
                ShowPassingFlash(
                    static_curve.copy().set_stroke(flash_color, width=width),
                    run_time=2
                )
            )
            # We never .add() static_curve, so it's not left on screen.

        # Show the flash
        flash_residual_poly(at_alpha=0)

        monster_label_1 = Tex(r"Den lineære model giver \\ et tydeligt mønster i residualerne!", color=BLACK, font_size=24).move_to(residual_dots[len(x_data)-1].get_center()+2.2*RIGHT+0.2*UP)
        self.play(Write(monster_label_1))
        self.wait(3)
        self.play(FadeOut(monster_label_1))

        # --------------------------------------------------------
        # 9) Animate alpha 0->1->2->3, flashing after the first two transitions
        # --------------------------------------------------------
        # 0 -> 1
        self.play(alpha_tracker.animate.set_value(1), run_time=3)
        flash_residual_poly(at_alpha=1)

        monster_label_2 = Tex(r"Den nye linære model giver \\ stadig mønster i residualerne!", color=BLACK, font_size=24).move_to(residual_dots[len(x_data)-1].get_center()+2*RIGHT)
        
        self.play(Write(monster_label_2))
        self.wait(3)
        self.play(FadeOut(monster_label_2))
        self.wait(1)
        
        description_text = Tex(r"En mere avanceret model kunne her \\ fjerne mønstret fra residualerne", color=BLACK, font_size=24).move_to(residual_dots[int(len(x_data)/2)].get_center()+2*UP)

        # 1 -> 2
        self.play(alpha_tracker.animate.set_value(2), run_time=3)
        self.play(Write(description_text))
        self.wait(3)
        self.play(FadeOut(description_text))
        self.wait(4)

### Residualerne skal have konstant varians

Når vi betragter residualplottet, så skal der gerne være symmetri i residualerne rundt om $x$-aksen. Vi vil gerne have, at residualerne varierer konstant rundt om $x$-aksen og størrelsen af variationen skal være nogenlunde ens på tværs af alle observationerne $x_i$. Dette kaldes homoskedacitet. Hvis variansen af residualerne stiger eller falder systematisk med $x_i$, så vil det kunne ses som en tragt-form i residualplottet. Er dette tilfældet, så vil vi muligvis have brug for en anden model eller tage højde for den varierende varians på en anden måde.

In [10]:
from manim import *

In [11]:
%%manim -v WARNING -qm --format=mp4 S3_FunnelResidualPlot

# Config manim
config.media_embed = True
config.media_width = "100%"

class S3_FunnelResidualPlot(Scene):
    def construct(self):
        self.camera.background_color = WHITE

        # --------------------------------------------------------
        # 1) Data: 60 punkter for tæt sky
        # --------------------------------------------------------
        np.random.seed(42)
        x_data = np.linspace(0, 10, 70)   # 60 punkter
        r_data = 2 * np.random.normal(0, 1, len(x_data))  # Residualer ~ N(0,1)

        # --------------------------------------------------------
        # 2) Ekstrem akseopsætning, da vi kan nå 10x => ±30..±40..±50
        # --------------------------------------------------------
        x_min, x_max = 0, 10
        x_range = [x_min, x_max+1, 1]
        y_range = [-50, 50, 10]  # MASSIV range til at vise 10× spredning

        axis_config = {
            "color": BLACK,
            "include_numbers": True,
            "include_ticks": True,
            "tick_size": 0.05,
            "stroke_width": 1,
        }

        axes = Axes(
            x_range=x_range,
            y_range=y_range,
            x_length=8,
            y_length=6,  # Stor akse
            tips=True,
            axis_config=axis_config
        )
        axes.move_to(ORIGIN)

        axis_labels = axes.get_axis_labels(
            x_label=MathTex("x", color=BLACK),
            y_label=MathTex("z", color=BLACK)
        )

        #self.play(Create(axes))
        #self.play(FadeIn(axis_labels))

        self.add(axes, axis_labels)

        # --------------------------------------------------------
        # 3) Én ValueTracker alpha i [0..4], stykvis:
        #    - [0..1] : normal -> venstre tragt
        #    - [1..2] : venstre tragt -> normal
        #    - [2..3] : normal -> højre tragt
        #    - [3..4] : højre tragt -> normal
        # --------------------------------------------------------
        alpha_tracker = ValueTracker(0.0)

        # Vi skruer op til 10× skalering:
        # - Venstre tragt: x=0 => 10, x=10 => 1 => scale = 10 - 0.9*x
        # - Højre  tragt: x=10 => 10, x=0 => 1 => scale = 1 + 0.9*x

        def scale_factor(alpha, x):
            left_scale  = 10 - 0.9*x  # x=0 => 10, x=10 => 1
            right_scale = 1 + 0.9*x   # x=0 => 1,  x=10 => 10

            if 0 <= alpha < 1:
                # normal(1) -> venstre tragt
                t = alpha
                return 1*(1 - t) + left_scale*t
            elif 1 <= alpha < 2:
                # venstre tragt -> normal
                t = alpha - 1
                return left_scale*(1 - t) + 1*t
            elif 2 <= alpha < 3:
                # normal -> højre tragt
                t = alpha - 2
                return 1*(1 - t) + right_scale*t
            else:
                # højre tragt -> normal
                t = alpha - 3
                return right_scale*(1 - t) + 1*t

        def funnel_residual(i):
            alpha = alpha_tracker.get_value()
            return r_data[i] * scale_factor(alpha, x_data[i])

        # --------------------------------------------------------
        # 4) Punkter (always_redraw)
        # --------------------------------------------------------
        dots = VGroup()
        for i in range(len(x_data)):
            def make_dot(i=i):
                x_val = x_data[i]
                z_val = funnel_residual(i)
                return Dot(
                    axes.coords_to_point(x_val, z_val),
                    color=animat_red,
                    radius=0.06
                )
            dot_obj = always_redraw(make_dot)
            dots.add(dot_obj)

        #self.play(*[FadeIn(d) for d in dots], run_time=2)
        self.add(*[d for d in dots])
        self.wait(1)

        # --------------------------------------------------------
        # 5) Konvekse buer => vild form (±50 i yderpunkter)
        # --------------------------------------------------------
        # Venstre tragt: stor amplitude ved x=0 => ±50, ved x=10 => ±10
        #   => top_left(x)  = +0.5*(x-10)^2 + 10
        #      x=10 => 10, x=0 => +0.5*100+10=60  (Vi kan så evt. tage 50)
        #
        # Justerer for at få ~50 ved x=0 => top_left(0)= 0.4*100 + 10=50
        # => 0.4*(x-10)^2 + 10
        # => x=10 => 10, x=0 => 0.4*100+10=50
        #
        # Samme for bund, men negativ:
        #   bot_left(x) = -0.4*(x-10)^2 - 10 => x=10 => -10, x=0 => -50
        #
        # Højre tragt: stor amplitude ved x=10 => ±50, ved x=0 => ±10
        #   => top_right(x) = 0.4*(x-0)^2 + 10 => x=0=>10, x=10=> 0.4*100+10=50
        #   => bot_right(x)= -0.4*(x-0)^2 -10 => x=0=>-10,x=10=>-50
        #
        # stroke_width=7 => ret markant streg.

        def top_left(x):
            return 0.3*(x - 10)**2 + 15
        def bot_left(x):
            return -0.3*(x - 10)**2 - 15

        def top_right(x):
            return 0.3*(x - 0)**2 + 15
        def bot_right(x):
            return -0.3*(x - 0)**2 - 15

        arc_left_top = axes.plot(top_left,  x_range=[0,10], color=animat_green, stroke_width=4)
        arc_left_bot = axes.plot(bot_left,  x_range=[0,10], color=animat_green, stroke_width=4)
        arc_right_top = axes.plot(top_right, x_range=[0,10], color=animat_green, stroke_width=4)
        arc_right_bot = axes.plot(bot_right, x_range=[0,10], color=animat_green, stroke_width=4)

        funnel_form_1 = Tex(r"Residualerne har tragt-form fra venstre!", color=BLACK, font_size=24).move_to(arc_left_top.get_center()+2*RIGHT)
        funnel_form_2 = Tex(r"Residualerne har tragt-form fra højre!", color=BLACK, font_size=24).move_to(arc_right_bot.get_center()+LEFT+0.7*DOWN)

        # --------------------------------------------------------
        # 6) Animation
        # --------------------------------------------------------
        # (0->1) Venstre tragt => vis buer => fjern buer
        self.play(alpha_tracker.animate.set_value(1), run_time=2)
        self.play(Create(arc_left_top), Create(arc_left_bot), run_time=1)
        self.play(Write(funnel_form_1))
        self.wait(2)
        self.play(FadeOut(arc_left_top), FadeOut(arc_left_bot), FadeOut(funnel_form_1), run_time=1)

        # (1->2) => normal
        self.play(alpha_tracker.animate.set_value(2), run_time=2)
        self.wait(0.5)

        # (2->3) Højre tragt => vis buer => fjern buer
        self.play(alpha_tracker.animate.set_value(3), run_time=2)
        self.play(Create(arc_right_top), Create(arc_right_bot), run_time=1)
        self.play(Write(funnel_form_2))
        self.wait(2)
        self.play(FadeOut(arc_right_top), FadeOut(arc_right_bot), FadeOut(funnel_form_2), run_time=1)

        # (3->4) => normal
        self.play(alpha_tracker.animate.set_value(4), run_time=2)
        self.wait(3)


### Prøv selv

<iframe scrolling="no" title="LinearRegression" src="https://www.geogebra.org/material/iframe/id/yxh3nheb/width/0/height//border/888888/sfsb/true/smb/false/stb/false/stbh/false/ai/false/asb/false/sri/true/rc/false/ld/false/sdz/false/ctl/false" height="700px" style="border:0px;"> </iframe>